### Parsing X data

This example is to help illustrate the structure of the response data

**Notes**:
- The code is extremely slow due to constant calls to `.apply()`
- The GraphQL response data is not consistent, the following examples may not work in all cases.

In [1]:
import pandas as pd
from twitter.scraper import Scraper

scraper = Scraper(
    cookies={
        'auth_token':...,
        'ct0':...,
    }
)


### Likes

In [2]:
likes = scraper.likes([33836629],count=100,limit=100)

df_likes = (
    pd.json_normalize(likes, record_path=['data','user','result','timeline_v2','timeline','instructions'])
    ['entries']
    .explode()
    .apply(pd.Series)
    ['content']
    .apply(pd.Series)
    .pipe(lambda df:df[df['__typename']!='TimelineTimelineCursor'])
    ['itemContent']
    .apply(pd.Series)
    ['tweet_results']
    .apply(pd.Series)
    ['result']
    .apply(pd.Series)
    .pipe(lambda df:df[df['__typename']!='TweetWithVisibilityResults'])
    ['legacy']
    .apply(pd.Series)
    .assign(created_at=lambda x:pd.to_datetime(x['created_at']))
    .sort_values('created_at', ascending=False)
    .reset_index(drop=True)
    [[
      'user_id_str',
      'id_str',
      'created_at',
      'bookmark_count',
      'favorite_count',
      'full_text',
      'possibly_sensitive',
      'quote_count',
      'reply_count',
      'retweet_count'
    ]]
)
df_likes



Likes: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.40s/it]
/tmp/ipykernel_11828/237363432.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(created_at=lambda x:pd.to_datetime(x['created_at']))


,user_id_str,id_str,created_at,bookmark_count,favorite_count,full_text,possibly_sensitive,quote_count,reply_count,retweet_count
0,1376951872356024325,1776320004465582331,2024-04-05 18:44:26+00:00,248,338,Schedule-Free Learning\nhttps://t.co/HEl6dOcT2...,False,10,13,56
1,3278906401,1776271459352228062,2024-04-05 15:31:32+00:00,4,74,We’ve teamed up with PayPal to offer MetaMask ...,NaN,0,6,3
2,4558314927,1776232714363871586,2024-04-05 12:57:34+00:00,29,309,"I like to think of myself as a researcher, but...",NaN,2,7,7
3,13235832,1776080884019700108,2024-04-05 02:54:15+00:00,40,932,Someone should make a leaderboard for how freq...,NaN,11,47,58
4,1703117848930099200,1776041561710837833,2024-04-05 00:18:00+00:00,93,311,"""Young woman with blonde hair wearing a backpa...",NaN,8,40,11
...,...,...,...,...,...,...,...,...,...,...
112,34792369,1740880412795379785,2023-12-29 23:40:09+00:00,10,377,Dive into the best of 2023 with Porch Potty on...,False,1,13,99
113,2329526714,1739958754219631031,2023-12-27 10:37:48+00:00,1078,20857,What happened in 1970? \n\nNixon signed the 19...,False,357,739,7573
114,5778712,1699446703857045540,2023-09-06 15:37:23+00:00,135,784,another way of thinking about this — much of m...,False,14,46,34
115,18570772,1642044121341919232,2023-04-01 06:00:00+00:00,580,11854,"As Beyoncé once said, “If you like it, then yo...",NaN,123,777,2321


### Tweet Details

In [3]:
tweets_details = scraper.tweets_details([1770164518758633590], count=100, limit=100)

df_tweets_details = (
    pd.json_normalize(tweets_details,record_path=['data','threaded_conversation_with_injections_v2','instructions'])
    ['entries']
    .dropna()
    .explode()
    .apply(pd.Series)
    ['content']
    .apply(pd.Series)
    ['items']
    .dropna()
    .explode()
    .apply(pd.Series)
    ['item']
    .apply(pd.Series)
    ['itemContent']
    .apply(pd.Series)
    .pipe(lambda df:df[df['__typename']!='TimelineTimelineCursor'])
    ['tweet_results']
    .apply(pd.Series)
    ['result']
    .apply(pd.Series)
    .pipe(lambda df:df[df['__typename']!='TweetWithVisibilityResults'])
    ['legacy']
    .apply(pd.Series)
    .pipe(lambda x: pd.concat([x, x['entities'].apply(pd.Series)],axis=1))
    .assign(created_at=lambda x:pd.to_datetime(x['created_at']))
    .sort_values('created_at', ascending=False)
    .reset_index(drop=True)
    .drop('entities',axis=1)
    [[
    'created_at',
    'id_str',
    'user_id_str',
    'full_text',
    'favorite_count',
    'urls',
    'hashtags',
    'user_mentions',
    ]]

)
df_tweets_details


TweetDetail: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.06s/it]
/tmp/ipykernel_11828/4204966796.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(created_at=lambda x:pd.to_datetime(x['created_at']))


,created_at,id_str,user_id_str,full_text,favorite_count,urls,hashtags,user_mentions
0,2024-04-05 16:27:24+00:00,1776285521603363107,307810043,Integrate Advanced Image and Video Customizati...,0,[],"[{'indices': [105, 112], 'text': 'nodejs'}, {'...",[]
1,2024-04-05 16:27:24+00:00,1776285521569865904,307810043,Integrate Advanced Image and Video Customizati...,0,[],"[{'indices': [105, 112], 'text': 'nodejs'}, {'...",[]
2,2024-04-05 16:27:24+00:00,1776285521569865904,307810043,Integrate Advanced Image and Video Customizati...,0,[],"[{'indices': [105, 112], 'text': 'nodejs'}, {'...",[]
3,2024-04-05 16:27:24+00:00,1776285521603363107,307810043,Integrate Advanced Image and Video Customizati...,0,[],"[{'indices': [105, 112], 'text': 'nodejs'}, {'...",[]
4,2024-04-04 18:39:56+00:00,1775956485324849319,1594005431164321793,Embark on a magical morning adventure with eve...,85,[],[],[]
5,2024-04-01 17:24:10+00:00,1774850254468436277,1773621435199586304,@karpathy @__tinygrad__ @realGeorgeHotz Recent...,0,[],[],"[{'id_str': '33836629', 'name': 'Andrej Karpat..."
6,2024-04-01 02:46:28+00:00,1774629375121539126,1371119512313950210,@karpathy @__tinygrad__ @realGeorgeHotz yeah I...,0,[],[],"[{'id_str': '33836629', 'name': 'Andrej Karpat..."
7,2024-03-31 00:20:12+00:00,1774230175238197456,1316295394146410497,@karpathy @__tinygrad__ @realGeorgeHotz Colab ...,0,[],[],"[{'id_str': '33836629', 'name': 'Andrej Karpat..."
8,2024-03-29 23:47:06+00:00,1773859460139855938,911238967030403073,@karpathy @__tinygrad__ @realGeorgeHotz I was ...,0,[],[],"[{'id_str': '33836629', 'name': 'Andrej Karpat..."
9,2024-03-29 01:23:42+00:00,1773521382187717067,1231466530375970816,@karpathy @__tinygrad__ @realGeorgeHotz 👀,0,[],[],"[{'id_str': '33836629', 'name': 'Andrej Karpat..."


### Favoriters

In [4]:
favoriters = scraper.favoriters([1770164518758633590], count=100, limit=100)

df_favoriters = (
    pd.json_normalize(favoriters,record_path=['data','favoriters_timeline','timeline', 'instructions'])
    ['entries']
    .explode()
    .apply(pd.Series)
    ['content']
    .apply(pd.Series)
    .pipe(lambda df:df[df['__typename']!='TimelineTimelineCursor'])
    ['itemContent']
    .apply(pd.Series)
    ['user_results']
    .apply(pd.Series)
    ['result']
    .apply(pd.Series)
    .pipe(lambda x: pd.concat([x[['rest_id']],x['legacy'].apply(pd.Series)],axis=1))
    .dropna(subset='created_at')
    .assign(created_at=lambda x:pd.to_datetime(x['created_at']))
    .sort_values('created_at', ascending=False)
    .reset_index(drop=True)
    [[
      'rest_id',
      'created_at',
      'screen_name',
      'location',
      'followers_count',
      'statuses_count',
      'favourites_count',
      'media_count',
      'statuses_count'
    ]]
)
df_favoriters

Favoriters: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.22s/it]
/tmp/ipykernel_11828/1064589959.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(created_at=lambda x:pd.to_datetime(x['created_at']))


,rest_id,created_at,screen_name,location,followers_count,statuses_count,favourites_count,media_count,statuses_count
0,1774149870552580096,2024-03-30 19:01:21+00:00,k_weissgerber,,4.0,0.0,18.0,0.0,0.0
1,1772830183797440513,2024-03-27 03:37:25+00:00,kategerrald,,32.0,173.0,408.0,0.0,173.0
2,1772286724577837056,2024-03-25 15:37:39+00:00,DaringDott89944,,0.0,1.0,56.0,0.0,1.0
3,1771785767729389568,2024-03-24 06:27:00+00:00,ImeranceL33140,,2.0,1.0,60.0,0.0,1.0
4,1771782246837678080,2024-03-24 06:13:01+00:00,UsHarper77813,,2.0,1.0,66.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
163,15040790,2008-06-07 19:23:56+00:00,ssaroha,"San Jose, CA",80.0,599.0,1777.0,6.0,599.0
164,14578396,2008-04-29 01:12:13+00:00,tim_waggoner,Planet Earth,495.0,332.0,337.0,16.0,332.0
165,6452692,2007-05-30 20:13:53+00:00,goodwintercrop,,338.0,972.0,3874.0,48.0,972.0
166,5141381,2007-04-18 17:46:19+00:00,hangel,"Medellin, Colombia",493.0,9853.0,16483.0,669.0,9853.0
